In [ ]:
# 

## Serverless: AWS Lambda 
We want to build a web service that
- Gets the URL in the request
- Loads the image from this URL
- Uses TensorFlow Lite to apply the model to the image and get the predictions
- Responds with the results (figure 8.1)

To create this service, we will need to

- Convert the model from Keras to the TensorFlow Lite format
- Preprocess the images — resize them and apply the preprocessing function
- Package the code in a Docker image, and upload it to ECR (the Docker registry from AWS)
- Create and test the lambda function on AWS
- Make the lambda function available to everyone with AWS API Gateway

## Converting the model to TF Lite format

In [2]:
## lets turn the model to TF Lite format
!python convert.py

2022-09-01 09:11:51.919068: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-01 09:11:51.919116: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-01 09:11:54.588394: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-01 09:11:54.588439: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-01 09:11:54.588469: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora): /proc/driver/nvidia/version does not exist
2022-09-01 09:11:54.588705: I tensorflow/core/platform/cpu_feature_gua

## Preparing the images

In [4]:
from keras_image_helper import create_preprocessor

## lets load the Xception model with a target size
preprocessor = create_preprocessor('xception', target_size=(299, 299))


In [5]:
## lets get an image and prepare it
image_url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(image_url)

## Using the TensorFlow Lite model

In [8]:
import tflite_runtime.interpreter as tflite

## lets load the tflite model

## create the tf lite interpreter
interpreter = tflite.Interpreter('clothing-model-v4.tflite')

## initializes the interpreter with the model
interpreter.allocate_tensors()

##
input_details = interpreter.get_input_details()
input_index = input_details[0]['index']

## 
output_details = interpreter.get_output_details()
output_index = output_details[0]['index']

##
interpreter.set_tensor(input_index, X)
interpreter.invoke()

##
preds = interpreter.get_tensor(output_index)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [9]:
labels = [
'dress',
'hat',
'longsleeve',
'outwear',
'pants',
'shirt',
'shoes',
'shorts',
'skirt',
't-shirt'
]

results = dict(zip(labels, preds[0]))

In [10]:
results

{'dress': -1.5640455,
 'hat': -4.702159,
 'longsleeve': -0.96891385,
 'outwear': -1.591752,
 'pants': 9.307265,
 'shirt': -1.4798162,
 'shoes': -2.7068253,
 'shorts': 3.1710477,
 'skirt': -2.3949814,
 't-shirt': -3.6138039}

## Code for the lambda function